# 챕터 5 (OpenAI 버전): Landing Page Guide Skill 예제 구현

## 5-1. Landing Page Guide Skill 개요

OpenAI GPT-4o-mini 모델을 사용하여 실제 프로젝트에서 사용할 수 있는 랜딩 페이지 생성 Skill을 구현한다.

이 Skill은 다음 기능을 제공한다:
- 현대적인 랜딩 페이지 HTML/CSS/JavaScript 생성
- 반응형 디자인
- SEO 최적화
- 접근성 고려
- 다양한 섹션 구성

**OpenAI API 사용의 장점:**
- 빠른 응답 속도
- 비용 효율성 (gpt-4o-mini)
- Function Calling 지원
- JSON 모드 지원

In [ ]:
# 필요한 라이브러리 설치
%pip install -q openai pydantic python-dotenv pyyaml -q

In [1]:
from dotenv import load_dotenv
import os

MODEL = "gpt-4o-mini"
load_dotenv()

True

In [2]:
import openai
import os
from pathlib import Path
import yaml
from typing import Dict, List, Optional
from pydantic import BaseModel, Field
import json

# OpenAI 클라이언트 초기화
client = openai.OpenAI()

print("환경 설정 완료")
print(f"OpenAI 라이브러리 버전: {openai.__version__}")

환경 설정 완료
OpenAI 라이브러리 버전: 2.6.1


## 5-2. Landing Page Guide Skill 정의

전문적인 랜딩 페이지를 생성하는 Skill을 작성한다.

In [3]:
# skills 디렉토리 생성
skills_dir = Path("skills/landing_openai")
skills_dir.mkdir(exist_ok=True)

# Landing Page Guide Skill 내용
landing_page_skill_content = """# 랜딩 페이지 전문가 Skill

## 역할
현대적이고 전환율이 높은 랜딩 페이지를 설계하고 코드로 구현하는 전문가다.

## 핵심 원칙

### 1. 디자인 원칙
- **명확한 가치 제안**: 첫 화면에서 핵심 가치를 즉시 전달한다
- **시각적 계층**: 중요한 정보를 시각적으로 강조한다
- **여백 활용**: 충분한 여백으로 가독성을 높인다
- **일관된 브랜딩**: 색상, 폰트, 톤앤매너를 일관되게 유지한다

### 2. 구조 원칙
랜딩 페이지는 다음 섹션으로 구성한다:

1. **Hero Section (히어로 섹션)**
   - 강력한 헤드라인
   - 명확한 서브헤드라인
   - 눈에 띄는 CTA 버튼
   - 관련 이미지나 동영상

2. **Features Section (기능 섹션)**
   - 3-6개의 핵심 기능
   - 각 기능에 아이콘과 설명
   - 혜택 중심으로 작성

3. **Social Proof (사회적 증거)**
   - 고객 후기
   - 신뢰할 수 있는 기업 로고
   - 통계 및 성과 지표

4. **How It Works (작동 방식)**
   - 3-4단계로 설명
   - 간단하고 명확한 프로세스
   - 시각적 요소 활용

5. **Pricing (가격)**
   - 명확한 가격 정보
   - 옵션별 비교
   - 추천 플랜 강조

6. **FAQ (자주 묻는 질문)**
   - 일반적인 우려사항 해소
   - 5-8개의 질문과 답변

7. **Final CTA (최종 행동 유도)**
   - 강력한 재촉구
   - 긴급성이나 희소성 활용

### 3. 기술 구현 원칙

#### HTML 구조
- 시맨틱 HTML5 태그 사용 (header, nav, main, section, article, footer)
- 접근성을 위한 ARIA 속성 포함
- SEO를 위한 메타 태그 완비

#### CSS 스타일링
- 모바일 우선 반응형 디자인
- CSS Grid와 Flexbox 활용
- CSS 변수로 테마 관리
- 부드러운 애니메이션과 전환 효과
- 다크모드 지원 고려

#### JavaScript 기능
- 스크롤 애니메이션
- 폼 검증
- 스무스 스크롤
- 모바일 메뉴 토글
- 성능 최적화 (Lazy Loading 등)

### 4. 성능 최적화
- 이미지 최적화 (WebP, lazy loading)
- CSS/JS 최소화
- Critical CSS 인라인 처리
- 폰트 최적화

### 5. SEO 최적화
- 적절한 title과 meta description
- Open Graph 태그
- Schema.org 구조화된 데이터
- 명확한 heading 계층 (h1, h2, h3)

## 코드 출력 형식

### 단일 파일 HTML
모든 CSS와 JavaScript를 포함한 완전한 HTML 파일을 생성한다:

```html
<!DOCTYPE html>
<html lang="ko">
<head>
    <!-- 메타 태그 -->
    <style>
        /* CSS 코드 */
    </style>
</head>
<body>
    <!-- HTML 구조 -->
    <script>
        // JavaScript 코드
    </script>
</body>
</html>
```

### 코드 품질
- 주석으로 섹션을 명확히 구분한다
- 일관된 들여쓰기와 네이밍 컨벤션을 사용한다
- 재사용 가능한 컴포넌트 구조로 작성한다

## 응답 스타일
- 문장은 ~다 체를 사용한다
- 기술적으로 정확하고 실용적인 조언을 제공한다
- 실제 동작하는 완전한 코드를 생성한다
- 커스터마이징 가이드를 함께 제공한다
"""

# YAML 메타데이터 생성
landing_page_yaml = {
    'name': 'landing_page_guide',
    'version': '1.0.0',
    'description': '고전환율 랜딩 페이지 생성 전문가',
    'author': 'OpenAI Tutorial',
    'tags': ['web', 'landing-page', 'html', 'css', 'javascript', 'marketing'],
    'config': {
        'model': MODEL,
        'max_tokens': 8000,
        'temperature': 0.7
    },
    'content': landing_page_skill_content
}

landing_page_yaml_path = skills_dir / "landing_page_guide.yaml"
with open(landing_page_yaml_path, "w", encoding="utf-8") as f:
    yaml.dump(landing_page_yaml, f, allow_unicode=True, default_flow_style=False)

print(f"Landing Page YAML 생성: {landing_page_yaml_path}")

Landing Page YAML 생성: skills/landing_openai/landing_page_guide.yaml


## 5-3. Pydantic 모델로 랜딩 페이지 요구사항 정의

In [4]:
class LandingPageRequest(BaseModel):
    """
    랜딩 페이지 생성 요청 모델
    """
    product_name: str = Field(..., description="제품/서비스 이름")
    product_description: str = Field(..., description="제품/서비스 설명")
    target_audience: str = Field(..., description="타겟 고객")
    key_features: List[str] = Field(
        ...,
        min_length=3,
        max_length=6,
        description="핵심 기능 목록"
    )
    color_scheme: Optional[str] = Field(
        default="모던한 파란색과 흰색",
        description="색상 스킴"
    )
    brand_tone: Optional[str] = Field(
        default="전문적이고 신뢰감 있는",
        description="브랜드 톤앤매너"
    )

class LandingPageResponse(BaseModel):
    """
    랜딩 페이지 생성 응답 모델
    """
    html_content: str = Field(..., description="생성된 HTML 코드")
    file_size: int = Field(..., description="파일 크기 (문자 수)")
    sections_included: List[str] = Field(
        default_factory=list,
        description="포함된 섹션 목록"
    )
    customization_tips: Optional[str] = Field(
        default=None,
        description="커스터마이징 팁"
    )

print("Pydantic 모델 정의 완료")

Pydantic 모델 정의 완료


## 5-4. OpenAI 기반 LandingPageAgent 구현

In [5]:
class OpenAILandingPageAgent:
    """
    OpenAI API를 사용하는 랜딩 페이지 생성 에이전트
    """
    
    def __init__(self, api_key: Optional[str] = None):
        """
        에이전트를 초기화한다
        
        Args:
            api_key: OpenAI API 키
        """
        self.client = openai.OpenAI()
        self.skill_content = None
        self.skill_config = None
    
    def load_skill(self, skill_path: str) -> None:
        """
        Skill 파일을 로드한다
        
        Args:
            skill_path: Skill YAML 파일 경로
        """
        with open(skill_path, 'r', encoding='utf-8') as f:
            skill_data = yaml.safe_load(f)
        
        self.skill_content = skill_data['content']
        self.skill_config = skill_data.get('config', {})
        
        print(f"Skill 로드 완료: {skill_data['name']} v{skill_data['version']}")
        print(f"모델: {self.skill_config.get('model', MODEL)}")
    
    def generate_landing_page(
        self,
        request: LandingPageRequest
    ) -> LandingPageResponse:
        """
        랜딩 페이지를 생성한다
        
        Args:
            request: 랜딩 페이지 요청
            
        Returns:
            생성된 랜딩 페이지 응답
        """
        # 프롬프트 구성
        features_text = "\n".join([
            f"- {feature}" for feature in request.key_features
        ])
        
        user_prompt = f"""
다음 정보를 바탕으로 완전한 랜딩 페이지를 만들어줘:

**제품명**: {request.product_name}

**제품 설명**: {request.product_description}

**타겟 고객**: {request.target_audience}

**핵심 기능**:
{features_text}

**색상 스킴**: {request.color_scheme}

**브랜드 톤**: {request.brand_tone}

요구사항:
1. 모든 CSS와 JavaScript를 포함한 단일 HTML 파일을 생성한다
2. 반응형 디자인을 적용한다 (모바일, 태블릿, 데스크톱)
3. Hero, Features, How It Works, FAQ, CTA 섹션을 포함한다
4. 실제 동작하는 완전한 코드를 작성한다
5. 주석으로 각 섹션을 명확히 표시한다
6. 접근성(ARIA)과 SEO를 고려한다
7. 부드러운 스크롤 애니메이션을 추가한다

HTML 코드만 출력하고, 추가 설명은 포함하지 말아줘.
"""
        
        # OpenAI API 호출
        print(f"\n랜딩 페이지 생성 중... (모델: {self.skill_config.get('model', MODEL)})")
        
        response = self.client.chat.completions.create(
            model=self.skill_config.get('model', MODEL),
            max_tokens=self.skill_config.get('max_tokens', 8000),
            temperature=self.skill_config.get('temperature', 0.7),
            messages=[
                {
                    "role": "system",
                    "content": self.skill_content
                },
                {
                    "role": "user",
                    "content": user_prompt
                }
            ]
        )
        
        html_content = response.choices[0].message.content
        
        # HTML 코드 블록에서 실제 HTML 추출
        if '```html' in html_content:
            start = html_content.find('```html') + 7
            end = html_content.rfind('```')
            html_content = html_content[start:end].strip()
        elif '```' in html_content:
            start = html_content.find('```') + 3
            end = html_content.rfind('```')
            html_content = html_content[start:end].strip()
        
        # 응답 생성
        return LandingPageResponse(
            html_content=html_content,
            file_size=len(html_content),
            sections_included=self._detect_sections(html_content)
        )
    
    def _detect_sections(self, html: str) -> List[str]:
        """
        HTML에서 포함된 섹션을 감지한다
        
        Args:
            html: HTML 콘텐츠
            
        Returns:
            섹션 이름 리스트
        """
        sections = []
        html_lower = html.lower()
        
        section_keywords = {
            'Hero': ['hero', 'main-header', 'jumbotron'],
            'Features': ['features', 'benefits'],
            'How It Works': ['how-it-works', 'process', 'steps'],
            'Pricing': ['pricing', 'plans'],
            'FAQ': ['faq', 'questions'],
            'CTA': ['cta', 'call-to-action'],
            'Social Proof': ['testimonial', 'reviews', 'social-proof']
        }
        
        for section_name, keywords in section_keywords.items():
            if any(keyword in html_lower for keyword in keywords):
                sections.append(section_name)
        
        return sections
    
    def save_landing_page(
        self,
        response: LandingPageResponse,
        output_path: str
    ) -> None:
        """
        생성된 랜딩 페이지를 파일로 저장한다
        
        Args:
            response: 랜딩 페이지 응답
            output_path: 저장할 파일 경로
        """
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(response.html_content)
        
        print(f"\n랜딩 페이지 저장 완료: {output_path}")
        print(f"파일 크기: {response.file_size:,} 문자")
        print(f"포함된 섹션: {', '.join(response.sections_included)}")

print("OpenAILandingPageAgent 클래스 정의 완료")

OpenAILandingPageAgent 클래스 정의 완료


## 5-5. 실전 예제 1: SaaS 제품 랜딩 페이지

In [6]:
# 에이전트 생성 및 Skill 로드
agent = OpenAILandingPageAgent()
agent.load_skill('skills/landing_openai/landing_page_guide.yaml')

print("\n" + "=" * 80)
print("예제 1: AI 기반 작문 도구 랜딩 페이지 생성")
print("=" * 80)

Skill 로드 완료: landing_page_guide v1.0.0
모델: gpt-4o-mini

예제 1: AI 기반 작문 도구 랜딩 페이지 생성


In [7]:
# 랜딩 페이지 요청 생성
request1 = LandingPageRequest(
    product_name="WriteMaster AI",
    product_description="AI가 도와주는 스마트 작문 도구. 블로그, 이메일, 보고서 작성을 10배 빠르게 완성한다.",
    target_audience="콘텐츠 크리에이터, 마케터, 비즈니스 전문가",
    key_features=[
        "AI 기반 문장 제안 및 자동 완성",
        "50개 이상의 글쓰기 템플릿",
        "실시간 문법 및 맞춤법 검사",
        "다양한 톤과 스타일 지원",
        "팀 협업 기능"
    ],
    color_scheme="모던한 파란색과 흰색 조합",
    brand_tone="전문적이고 혁신적인"
)

# 랜딩 페이지 생성
response1 = agent.generate_landing_page(request1)

print(f"\n생성 완료!")
print(f"HTML 길이: {response1.file_size:,} 문자")
print(f"포함된 섹션: {', '.join(response1.sections_included)}")


랜딩 페이지 생성 중... (모델: gpt-4o-mini)

생성 완료!
HTML 길이: 5,183 문자
포함된 섹션: Hero, Features, How It Works, FAQ, CTA


In [15]:
# 생성된 랜딩 페이지 저장
output_dir = Path("output_openai")
output_dir.mkdir(exist_ok=True)

output_path1 = output_dir / "writemaster_landing.html"
agent.save_landing_page(response1, str(output_path1))

print(f"\n파일 경로: {output_path1.absolute()}")
print("브라우저에서 열어서 확인할 수 있다")


랜딩 페이지 저장 완료: output_openai/writemaster_landing.html
파일 크기: 4,290 문자
포함된 섹션: Hero, Features, How It Works, FAQ, CTA

파일 경로: /Users/namuai/03-books/agent-skills/output_openai/writemaster_landing.html
브라우저에서 열어서 확인할 수 있다


## 5-6. 실전 예제 2: 모바일 앱 랜딩 페이지

In [16]:
print("\n" + "=" * 80)
print("예제 2: 피트니스 트래킹 앱 랜딩 페이지 생성")
print("=" * 80)

request2 = LandingPageRequest(
    product_name="FitTrack Pro",
    product_description="AI 개인 트레이너가 당신의 건강을 책임진다. 맞춤형 운동 계획과 영양 관리를 한 곳에서.",
    target_audience="운동을 시작하고 싶은 초보자, 체계적인 관리가 필요한 헬스 애호가",
    key_features=[
        "AI 기반 맞춤형 운동 계획",
        "실시간 자세 교정 및 피드백",
        "칼로리 및 영양소 추적",
        "진행 상황 시각화 대시보드",
        "커뮤니티 챌린지 참여",
        "웨어러블 기기 연동"
    ],
    color_scheme="활동적인 오렌지색과 검정색 조합",
    brand_tone="에너지 넘치고 동기부여하는"
)

response2 = agent.generate_landing_page(request2)

output_path2 = output_dir / "fittrack_landing.html"
agent.save_landing_page(response2, str(output_path2))

print(f"\n파일 경로: {output_path2.absolute()}")


예제 2: 피트니스 트래킹 앱 랜딩 페이지 생성

랜딩 페이지 생성 중... (모델: gpt-4o-mini)

랜딩 페이지 저장 완료: output_openai/fittrack_landing.html
파일 크기: 5,302 문자
포함된 섹션: Hero, Features, How It Works, FAQ, CTA

파일 경로: /Users/namuai/03-books/agent-skills/output_openai/fittrack_landing.html


## 5-7. 실전 예제 3: 이커머스 제품 랜딩 페이지

In [17]:
print("\n" + "=" * 80)
print("예제 3: 프리미엄 커피 구독 서비스 랜딩 페이지 생성")
print("=" * 80)

request3 = LandingPageRequest(
    product_name="Bean Journey",
    product_description="세계 각지의 스페셜티 커피를 매달 문 앞으로. 바리스타가 엄선한 원두로 집에서 즐기는 카페 경험.",
    target_audience="커피를 사랑하는 사람들, 새로운 맛을 탐험하고 싶은 호기심 많은 소비자",
    key_features=[
        "매달 다른 나라의 스페셜티 원두 배송",
        "바리스타의 추출 가이드 포함",
        "원두 농장 스토리와 테이스팅 노트",
        "로스팅 후 7일 이내 신선 배송",
        "언제든지 변경/취소 가능한 유연한 구독"
    ],
    color_scheme="따뜻한 갈색과 크림색 조합",
    brand_tone="친근하고 전문적인"
)

response3 = agent.generate_landing_page(request3)

output_path3 = output_dir / "beanjourney_landing.html"
agent.save_landing_page(response3, str(output_path3))

print(f"\n파일 경로: {output_path3.absolute()}")


예제 3: 프리미엄 커피 구독 서비스 랜딩 페이지 생성

랜딩 페이지 생성 중... (모델: gpt-4o-mini)

랜딩 페이지 저장 완료: output_openai/beanjourney_landing.html
파일 크기: 4,234 문자
포함된 섹션: Hero, Features, How It Works, FAQ, CTA

파일 경로: /Users/namuai/03-books/agent-skills/output_openai/beanjourney_landing.html


## 5-8. 생성된 파일 분석 및 비교

In [18]:
import re

def analyze_html_file(file_path: str) -> Dict:
    """
    생성된 HTML 파일을 분석한다
    
    Args:
        file_path: 분석할 HTML 파일 경로
        
    Returns:
        분석 결과 딕셔너리
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    # 기본 통계
    stats = {
        'file_name': Path(file_path).name,
        'file_size': len(content),
        'line_count': content.count('\n'),
    }
    
    # HTML 요소 분석
    stats['sections'] = content.count('<section')
    stats['has_hero'] = 'hero' in content.lower()
    stats['has_features'] = 'feature' in content.lower()
    stats['has_cta'] = 'cta' in content.lower() or 'call-to-action' in content.lower()
    stats['has_faq'] = 'faq' in content.lower()
    stats['has_pricing'] = 'pricing' in content.lower() or 'price' in content.lower()
    
    # CSS 분석
    stats['has_css'] = '<style' in content
    stats['has_responsive'] = '@media' in content
    stats['css_variables'] = '--' in content and ':root' in content
    
    # JavaScript 분석
    stats['has_javascript'] = '<script' in content
    stats['has_smooth_scroll'] = 'scroll' in content.lower() and 'smooth' in content.lower()
    
    # SEO 요소
    stats['has_meta_description'] = 'meta name="description"' in content
    stats['has_og_tags'] = 'og:' in content
    stats['has_title'] = '<title>' in content
    
    # 접근성
    stats['has_aria'] = 'aria-' in content
    stats['has_alt_text'] = 'alt="' in content
    
    return stats

print("\n" + "=" * 80)
print("생성된 랜딩 페이지 분석")
print("=" * 80)

all_stats = []
for output_file in output_dir.glob("*.html"):
    stats = analyze_html_file(str(output_file))
    all_stats.append(stats)
    
    print(f"\n파일: {stats['file_name']}")
    print(f"  크기: {stats['file_size']:,} 문자")
    print(f"  라인 수: {stats['line_count']:,}")
    print(f"  섹션 수: {stats['sections']}")
    
    print(f"  주요 요소:")
    print(f"    - Hero 섹션: {'O' if stats['has_hero'] else 'X'}")
    print(f"    - Features: {'O' if stats['has_features'] else 'X'}")
    print(f"    - CTA: {'O' if stats['has_cta'] else 'X'}")
    print(f"    - FAQ: {'O' if stats['has_faq'] else 'X'}")
    print(f"    - Pricing: {'O' if stats['has_pricing'] else 'X'}")
    
    print(f"  기술 요소:")
    print(f"    - CSS: {'O' if stats['has_css'] else 'X'}")
    print(f"    - 반응형: {'O' if stats['has_responsive'] else 'X'}")
    print(f"    - CSS 변수: {'O' if stats['css_variables'] else 'X'}")
    print(f"    - JavaScript: {'O' if stats['has_javascript'] else 'X'}")
    print(f"    - 스무스 스크롤: {'O' if stats['has_smooth_scroll'] else 'X'}")
    
    print(f"  SEO & 접근성:")
    print(f"    - Title: {'O' if stats['has_title'] else 'X'}")
    print(f"    - Meta Description: {'O' if stats['has_meta_description'] else 'X'}")
    print(f"    - Open Graph: {'O' if stats['has_og_tags'] else 'X'}")
    print(f"    - ARIA 속성: {'O' if stats['has_aria'] else 'X'}")
    print(f"    - Alt 텍스트: {'O' if stats['has_alt_text'] else 'X'}")


생성된 랜딩 페이지 분석

파일: writemaster_landing.html
  크기: 4,290 문자
  라인 수: 163
  섹션 수: 5
  주요 요소:
    - Hero 섹션: O
    - Features: O
    - CTA: O
    - FAQ: O
    - Pricing: X
  기술 요소:
    - CSS: O
    - 반응형: O
    - CSS 변수: O
    - JavaScript: O
    - 스무스 스크롤: O
  SEO & 접근성:
    - Title: O
    - Meta Description: O
    - Open Graph: X
    - ARIA 속성: X
    - Alt 텍스트: X

파일: beanjourney_landing.html
  크기: 4,234 문자
  라인 수: 155
  섹션 수: 4
  주요 요소:
    - Hero 섹션: O
    - Features: O
    - CTA: O
    - FAQ: O
    - Pricing: X
  기술 요소:
    - CSS: O
    - 반응형: O
    - CSS 변수: O
    - JavaScript: O
    - 스무스 스크롤: O
  SEO & 접근성:
    - Title: O
    - Meta Description: O
    - Open Graph: X
    - ARIA 속성: O
    - Alt 텍스트: X

파일: fittrack_landing.html
  크기: 5,302 문자
  라인 수: 174
  섹션 수: 4
  주요 요소:
    - Hero 섹션: O
    - Features: O
    - CTA: O
    - FAQ: O
    - Pricing: X
  기술 요소:
    - CSS: O
    - 반응형: O
    - CSS 변수: O
    - JavaScript: O
    - 스무스 스크롤: O
  SEO & 접근성:
    - Title: O
    - Meta Descrip

## 5-9. 커스터마이징 가이드 생성

In [19]:
# 커스터마이징 가이드 생성
customization_guide = """
# 랜딩 페이지 커스터마이징 가이드 (OpenAI 버전)

## 색상 변경하기
HTML 파일의 `<style>` 섹션 상단에 있는 CSS 변수를 수정한다:

```css
:root {
    --primary-color: #your-color;    /* 주 색상 */
    --secondary-color: #your-color;  /* 보조 색상 */
    --text-color: #your-color;       /* 텍스트 색상 */
    --bg-color: #your-color;         /* 배경 색상 */
}
```

## 텍스트 수정하기
1. Hero 섹션의 헤드라인과 서브헤드라인을 수정한다
2. Features 섹션에서 각 기능의 제목과 설명을 변경한다
3. CTA 버튼 텍스트를 목적에 맞게 수정한다

## 이미지 추가하기
```html
<img src="your-image.jpg" alt="설명" loading="lazy">
```

플레이스홀더 이미지를 실제 이미지로 교체한다.
추천 이미지 소스:
- Unsplash (unsplash.com)
- Pexels (pexels.com)
- Pixabay (pixabay.com)

## 폰트 변경하기
`<head>` 섹션에 Google Fonts를 추가한다:

```html
<link href="https://fonts.googleapis.com/css2?family=Your+Font&display=swap" rel="stylesheet">
```

CSS에서 폰트를 적용한다:
```css
body {
    font-family: 'Your Font', sans-serif;
}
```

## 섹션 추가/제거하기
1. 필요한 섹션을 찾는다 (주석으로 표시되어 있음)
2. 전체 `<section>` 태그를 복사하거나 삭제한다
3. ID와 클래스명을 중복되지 않게 수정한다

## 반응형 조정하기
미디어 쿼리를 수정하여 브레이크포인트를 조정한다:

```css
@media (max-width: 768px) {
    /* 태블릿 스타일 */
}

@media (max-width: 480px) {
    /* 모바일 스타일 */
}
```

## 애니메이션 수정하기
CSS 애니메이션 속도와 타이밍을 조정한다:

```css
transition: all 0.3s ease;  /* 속도를 0.3초에서 원하는 값으로 */
```

## SEO 최적화
1. `<title>` 태그 수정 (50-60자)
2. Meta description 작성 (150-160자)
3. Open Graph 태그 정보 업데이트
4. Alt 텍스트를 모든 이미지에 추가
5. 구조화된 데이터 추가 (Schema.org)

## 폼 연동하기
실제 이메일 수집을 위해 폼 액션을 설정한다:

```html
<form action="your-form-handler" method="POST" id="signup-form">
    <input type="email" name="email" required>
    <button type="submit">가입하기</button>
</form>
```

추천 서비스:
- Formspree (무료 티어 제공)
- Netlify Forms (Netlify 호스팅 시)
- Google Forms
- Mailchimp

## 분석 도구 추가하기
`</head>` 태그 직전에 Google Analytics 코드를 추가한다:

```html
<!-- Google Analytics -->
<script async src="https://www.googletagmanager.com/gtag/js?id=GA_MEASUREMENT_ID"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());
  gtag('config', 'GA_MEASUREMENT_ID');
</script>
```

## 성능 최적화
1. 이미지 압축 (TinyPNG, ImageOptim)
2. CSS/JS 최소화 (Minify)
3. Lazy loading 활용
4. CDN 사용 고려

## OpenAI 모델 활용 팁
- 더 긴 코드 생성이 필요하면 max_tokens 증가
- 더 창의적인 디자인을 원하면 temperature 증가 (0.8-0.9)
- 일관된 결과를 원하면 temperature 감소 (0.5-0.6)
- 특정 디자인 스타일을 요청할 수 있다 (미니멀, 모던, 클래식 등)
"""

guide_path = output_dir / "CUSTOMIZATION_GUIDE.md"
with open(guide_path, 'w', encoding='utf-8') as f:
    f.write(customization_guide)

print(f"커스터마이징 가이드 생성: {guide_path}")

커스터마이징 가이드 생성: output_openai/CUSTOMIZATION_GUIDE.md


## 5-10. 생성된 파일 목록 및 요약

In [20]:
print("\n" + "=" * 80)
print("생성된 모든 파일")
print("=" * 80)

print("\n[Skills]")
for file_path in sorted(skills_dir.glob("landing_page*")):
    size = file_path.stat().st_size
    print(f"  {file_path.name} ({size:,} bytes)")

print("\n[Output]")
for file_path in sorted(output_dir.glob("*")):
    size = file_path.stat().st_size
    print(f"  {file_path.name} ({size:,} bytes)")

# 통계 요약
if all_stats:
    print("\n" + "=" * 80)
    print("전체 통계")
    print("=" * 80)
    
    total_size = sum(s['file_size'] for s in all_stats)
    avg_size = total_size / len(all_stats)
    
    print(f"\n생성된 랜딩 페이지 수: {len(all_stats)}개")
    print(f"총 파일 크기: {total_size:,} 문자")
    print(f"평균 파일 크기: {avg_size:,.0f} 문자")
    
    # 기능 포함률
    features = ['has_hero', 'has_features', 'has_cta', 'has_faq', 'has_responsive', 'has_javascript']
    feature_names = ['Hero', 'Features', 'CTA', 'FAQ', '반응형', 'JavaScript']
    
    print("\n기능 포함률:")
    for feat, name in zip(features, feature_names):
        count = sum(1 for s in all_stats if s[feat])
        percentage = (count / len(all_stats)) * 100
        print(f"  {name}: {count}/{len(all_stats)} ({percentage:.0f}%)")


생성된 모든 파일

[Skills]
  landing_page_guide.yaml (3,591 bytes)

[Output]
  .ipynb_checkpoints (96 bytes)
  CUSTOMIZATION_GUIDE.md (3,269 bytes)
  beanjourney_landing.html (4,800 bytes)
  fittrack_landing.html (6,330 bytes)
  writemaster_landing.html (4,960 bytes)

전체 통계

생성된 랜딩 페이지 수: 3개
총 파일 크기: 13,826 문자
평균 파일 크기: 4,609 문자

기능 포함률:
  Hero: 3/3 (100%)
  Features: 3/3 (100%)
  CTA: 3/3 (100%)
  FAQ: 3/3 (100%)
  반응형: 3/3 (100%)
  JavaScript: 3/3 (100%)
